In [1]:
import numpy as np
import cv2
import joblib
import argparse
import time
import pickle
# from time import gmtime, strftime

In [2]:
def detect_face(img, faceCascade):
    faces = faceCascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(110, 110)
        #flags = cv2.CV_HAAR_SCALE_IMAGE
    )
    return faces

In [3]:
def calc_hist(img):
    histogram = [0] * 3
    for j in range(3):
        histr = cv2.calcHist([img], [j], None, [256], [0, 256])
        histr *= 255.0 / histr.max()
        histogram[j] = histr
    return np.array(histogram)

In [ ]:
import time
if __name__ == "__main__":
    l=dict()
    c=0
    
    clf = None
    try:
        clf = pickle.load(open("./model.pkl",'rb'))
    except IOError as e:
        print ("Error loading model <"+"./model.pkl"+">: {0}".format(e.strerror))
        exit(0)


    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print ("Error opening camera")
        exit(0)

    w = 320
    h = 240
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, h)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, w)
   
     

    filePath = "./haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(filePath)

    sample_number = 1
    count = 0
    measures = np.zeros(sample_number, dtype=np.float)

    while True:
        
        ret, img_bgr = cap.read()
        if ret is False:
            print("Error grabbing frame from camera")
            break

        img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

        faces = detect_face(img_gray, faceCascade)

        measures[count%sample_number]=0

        point = (0,0)
        for i, (x, y, w, h) in enumerate(faces):

            roi = img_bgr[y:y+h, x:x+w]
            c+=1
            img_ycrcb = cv2.cvtColor(roi, cv2.COLOR_BGR2YCR_CB)
            img_luv = cv2.cvtColor(roi, cv2.COLOR_BGR2LUV)

            ycrcb_hist = calc_hist(img_ycrcb)
            luv_hist = calc_hist(img_luv)

            feature_vector = np.append(ycrcb_hist.ravel(), luv_hist.ravel())
            feature_vector = feature_vector.reshape(1, len(feature_vector))

            prediction = clf.predict_proba(feature_vector)
            prob = prediction[0][1]

            measures[count % sample_number] = prob

            cv2.rectangle(img_bgr, (x, y), (x + w, y + h), (255, 0, 0), 2)

            point = (x, y-5)

            print(measures, np.mean(measures))
            if 0 not in measures:
                text = "Real"
                if np.mean(measures) >= 0.75:
                    text = "Fake"
                    font = cv2.FONT_HERSHEY_TRIPLEX
                    cv2.putText(img=img_bgr, text=text, org=point, fontFace=font, fontScale=0.9, color=(0, 0, 255),
                                thickness=2, lineType=cv2.LINE_AA)
                else:
                    #cv2.imshow("original",roi)
                    #time.sleep(10)
                    l.update({c:roi})
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(img=img_bgr, text=text, org=point, fontFace=font, fontScale=0.9,
                                color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

        count+=1
        cv2.imshow('img_rgb', img_bgr)

        key = cv2.waitKey(1)
        if key & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

c:\users\robin\desktop\pythonprojects\python365env\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\robin\desktop\pythonprojects\python365env\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\robin\desktop\pythonprojects\python365env\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator ExtraTreeC

[0.54012987] 0.5401298701298701
[0.72012987] 0.7201298701298702
[0.74012987] 0.7401298701298702
[0.74012987] 0.7401298701298702
[0.64012987] 0.6401298701298701
[0.64012987] 0.6401298701298701
[0.74012987] 0.7401298701298702
[0.74012987] 0.7401298701298702
[0.74012987] 0.7401298701298702
[0.64012987] 0.6401298701298701
[0.74012987] 0.7401298701298702
[0.74012987] 0.7401298701298702
[0.74012987] 0.7401298701298702
[0.72012987] 0.7201298701298702
[0.72012987] 0.7201298701298702
[0.64012987] 0.6401298701298701
[0.72012987] 0.7201298701298702
[0.64012987] 0.6401298701298701
[0.64012987] 0.6401298701298701
[0.72012987] 0.7201298701298702
[0.72012987] 0.7201298701298702
[0.72012987] 0.7201298701298702
[0.64012987] 0.6401298701298701
[0.64012987] 0.6401298701298701
[0.64012987] 0.6401298701298701
[0.62012987] 0.6201298701298701
[0.64012987] 0.6401298701298701
[0.74012987] 0.7401298701298702
[0.64012987] 0.6401298701298701
[0.62012987] 0.6201298701298701
[0.54012987] 0.5401298701298701
[0.54012

[0.74012987] 0.7401298701298702
[0.74012987] 0.7401298701298702
[0.64012987] 0.6401298701298701
[0.82012987] 0.82012987012987
[0.74012987] 0.7401298701298702
[0.74012987] 0.7401298701298702
[0.64727273] 0.6472727272727272
[0.72012987] 0.7201298701298702
[0.72012987] 0.7201298701298702
[0.74012987] 0.7401298701298702
[0.74012987] 0.7401298701298702
[0.64012987] 0.6401298701298701
[0.64727273] 0.6472727272727272
[0.72012987] 0.7201298701298702
[0.62727273] 0.6272727272727272
[0.54727273] 0.5472727272727272
[0.64727273] 0.6472727272727272
[0.64727273] 0.6472727272727272
[0.72727273] 0.7272727272727273
[0.64727273] 0.6472727272727272
[0.54727273] 0.5472727272727272
[0.74727273] 0.7472727272727273
[0.82012987] 0.82012987012987
[0.74012987] 0.7401298701298702
[0.74012987] 0.7401298701298702
[0.64012987] 0.6401298701298701
[0.84012987] 0.8401298701298702
[0.74012987] 0.7401298701298702
[0.52727273] 0.5272727272727272
[0.42727273] 0.42727272727272725
[0.62012987] 0.6201298701298701
[0.82170467

[0.59166667] 0.5916666666666666
[0.49166667] 0.4916666666666667
[0.48452381] 0.48452380952380947
[0.59166667] 0.5916666666666667
[0.59166667] 0.5916666666666666
[0.58452381] 0.5845238095238094
[0.49166667] 0.4916666666666666
[0.59166667] 0.5916666666666666
[0.59166667] 0.5916666666666666
[0.39166667] 0.39166666666666666


In [ ]:
cv2.imwrite('4.jpg', l[1])